In [1]:
from kavenegar import *
import json 
api = KavenegarAPI('4371523774647169686E725A492F3531522B59424E6B61516D6B6A584A55736B6E6B7775754A70636F356B3D')
params = {
    'sender': '20001551',
    'receptor': '09335999446',
    'message': 'سلام سفارش شما پردازش شد '
    
}   
response = api.sms_send(params)
print (str(response))



[{'messageid': 425138564, 'message': 'سلام سفارش شما پردازش شد ', 'status': 1, 'statustext': 'در صف ارسال', 'sender': '20001551', 'receptor': '09335999446', 'date': 1714314384, 'cost': 860}]


## From Excel

In [1]:
import pandas as pd
df = pd.read_excel("/Users/mohammad/Desktop/Book1.xlsx")


df["ردیف"] = df["ردیف"].fillna(method="ffill")
df.dropna(subset=["نام محصول"], inplace=True)
df = df.set_index("ردیف")
df = df.reset_index()
for i in df.index.to_list()[:-1]:
    row  = df.loc[i,"ردیف"]
    next_row = df.loc[i+1,"ردیف"]
    if next_row == row:
        df.loc[i+1,"تاریخ سفارش"] = df.loc[i,"تاریخ سفارش"]
        df.loc[i+1,"نام"] = df.loc[i,"نام"]
        df.loc[i+1,"نام خانوادگی"] = df.loc[i,"نام خانوادگی"]
        df.loc[i+1,"نام کافه"] = df.loc[i,"نام کافه"]
        df.loc[i+1,"شهر"] = df.loc[i,"شهر"]
        df.loc[i+1,"کد پیگیری"] = df.loc[i,"کد پیگیری"]


df.to_excel("hi.xlsx")

FileNotFoundError: [Errno 2] No such file or directory: '/Users/mohammad/Desktop/Book1.xlsx'

In [5]:
import streamlit as st
import pandas as pd
import datetime
from PIL import Image
import plotly.express as px
import plotly.graph_objects as go

# reading the data from excel file
df = pd.read_excel("/Users/mohammad/Downloads/Adidas.xlsx")
st.set_page_config(layout="wide")
st.markdown('<style>div.block-container{padding-top:1rem;}</style>', unsafe_allow_html=True)
image = Image.open('/Users/mohammad/Downloads/adidas-logo.jpg')

col1, col2 = st.columns([0.1,0.9])
with col1:
    st.image(image,width=100)

html_title = """
    <style>
    .title-test {
    font-weight:bold;
    padding:5px;
    border-radius:6px;
    }
    </style>
    <center><h1 class="title-test">Adidas Interactive Sales Dashboard</h1></center>"""
with col2:
    st.markdown(html_title, unsafe_allow_html=True)

col3, col4, col5 = st.columns([0.1,0.45,0.45])
with col3:
    box_date = str(datetime.datetime.now().strftime("%d %B %Y"))
    st.write(f"Last updated by:  \n {box_date}")

with col4:
    fig = px.bar(df, x = "Retailer", y = "TotalSales", labels={"TotalSales" : "Total Sales {$}"},
                 title = "Total Sales by Retailer", hover_data=["TotalSales"],
                 template="gridon",height=500)
    st.plotly_chart(fig,use_container_width=True)

_, view1, dwn1, view2, dwn2 = st.columns([0.15,0.20,0.20,0.20,0.20])
with view1:
    expander = st.expander("Retailer wise Sales")
    data = df[["Retailer","TotalSales"]].groupby(by="Retailer")["TotalSales"].sum()
    expander.write(data)
with dwn1:
    st.download_button("Get Data", data = data.to_csv().encode("utf-8"),
                       file_name="RetailerSales.csv", mime="text/csv")

df["Month_Year"] = df["InvoiceDate"].dt.strftime("%b'%y")
result = df.groupby(by = df["Month_Year"])["TotalSales"].sum().reset_index()

with col5:
    fig1 = px.line(result, x = "Month_Year", y = "TotalSales", title="Total Sales Over Time",
                   template="gridon")
    st.plotly_chart(fig1,use_container_width=True)

with view2:
    expander = st.expander("Monthly Sales")
    data = result
    expander.write(data)
with dwn2:
    st.download_button("Get Data", data = result.to_csv().encode("utf-8"),
                       file_name="Monthly Sales.csv", mime="text/csv")
    
st.divider()

result1 = df.groupby(by="State")[["TotalSales","UnitsSold"]].sum().reset_index()

# add the units sold as a line chart on a secondary y-axis
fig3 = go.Figure()
fig3.add_trace(go.Bar(x = result1["State"], y = result1["TotalSales"], name = "Total Sales"))
fig3.add_trace(go.Scatter(x=result1["State"], y = result1["UnitsSold"], mode = "lines",
                          name ="Units Sold", yaxis="y2"))
fig3.update_layout(
    title = "Total Sales and Units Sold by State",
    xaxis = dict(title="State"),
    yaxis = dict(title="Total Sales", showgrid = False),
    yaxis2 = dict(title="Units Sold", overlaying = "y", side = "right"),
    template = "gridon",
    legend = dict(x=1,y=1.1)
)
_, col6 = st.columns([0.1,1])
with col6:
    st.plotly_chart(fig3,use_container_width=True)

_, view3, dwn3 = st.columns([0.5,0.45,0.45])
with view3:
    expander = st.expander("View Data for Sales by Units Sold")
    expander.write(result1)
with dwn3:
    st.download_button("Get Data", data = result1.to_csv().encode("utf-8"), 
                       file_name = "Sales_by_UnitsSold.csv", mime="text/csv")
st.divider()

_, col7 = st.columns([0.1,1])
treemap = df[["Region","City","TotalSales"]].groupby(by = ["Region","City"])["TotalSales"].sum().reset_index()

def format_sales(value):
    if value >= 0:
        return '{:.2f} Lakh'.format(value / 1_000_00)

treemap["TotalSales (Formatted)"] = treemap["TotalSales"].apply(format_sales)

fig4 = px.treemap(treemap, path = ["Region","City"], values = "TotalSales",
                  hover_name = "TotalSales (Formatted)",
                  hover_data = ["TotalSales (Formatted)"],
                  color = "City", height = 700, width = 600)
fig4.update_traces(textinfo="label+value")

with col7:
    st.subheader(":point_right: Total Sales by Region and City in Treemap")
    st.plotly_chart(fig4,use_container_width=True)

_, view4, dwn4 = st.columns([0.5,0.45,0.45])
with view4:
    result2 = df[["Region","City","TotalSales"]].groupby(by=["Region","City"])["TotalSales"].sum()
    expander = st.expander("View data for Total Sales by Region and City")
    expander.write(result2)
with dwn4:
    st.download_button("Get Data", data = result2.to_csv().encode("utf-8"),
                                        file_name="Sales_by_Region.csv", mime="text.csv")

_,view5, dwn5 = st.columns([0.5,0.45,0.45])
with view5:
    expander = st.expander("View Sales Raw Data")
    expander.write(df)
with dwn5:
    st.download_button("Get Raw Data", data = df.to_csv().encode("utf-8"),
                       file_name = "SalesRawData.csv", mime="text/csv")
st.divider()

DeltaGenerator()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display

# Generate some sample data
x = np.linspace(0, 10, 100)
y = np.sin(x)

# Create a function to plot the data
def plot_data(amp, freq):
    plt.figure(figsize=(8, 6))
    plt.plot(x, amp * np.sin(freq * x))
    plt.title(f"Sine Wave: Amplitude={amp}, Frequency={freq}")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.grid(True)
    plt.show()

# Create sliders for amplitude and frequency
amp_slider = widgets.FloatSlider(value=1.0, min=0.1, max=2.0, step=0.1, description='Amplitude:')
freq_slider = widgets.FloatSlider(value=1.0, min=0.1, max=2.0, step=0.1, description='Frequency:')

# Create an interactive widget
interactive_plot = widgets.interactive(plot_data, amp=amp_slider, freq=freq_slider)

# Display the interactive widget
display(interactive_plot)
